# STAT 201 Group 1 Proposal

# Introduction

**TODO: insert background info on our question**

The question we will try to answer with our project is, 

Do more danceable and sadder songs stay popular for longer? 

We chose our two variables, danceability and song mood based on the research of **insert two papers here**. 

Our dataset includes all the songs that have been on the Top 200 Weekly (Global) charts of Spotify in 2020 & 2021. Each song is one observation within the dataset, and the columns are features of the songs, such as the highest position that a song has been in the Spotify Top 200 Weekly Global Charts in 2020 and 2021, the number of times a song has made this chart, the approximate number of streams the song has, the popularity of the track (between 0 and 100, with 100 being the most popular), the overall tempo of the track, and more. 

In particular, we are interested in the danceability and mood of the song, which in our dataset is called *valence*. **DESCRIBE HOW THEY'RE DESCRIBED IN THE DATASET**.   


# Preliminary Results

First, we read our data into R. **NOTE: DON'T USE UP A LOT OF SPACE**

In [1]:
# READ DATA

Next, we clean and wrangle it into a tidy format.

In [2]:
# WRANGLE

We plot our data. **TODO: describe how our plot addresses our question**

In [3]:
# PLOT

Now we compute estimates of **the parameter we identified across our groups** and present it in the form of a table.

In [ ]:
# TODO

# Methods

... 
hypotheses go here... 
.. so does boostrapping..
..asymptotics...

# Bibliography


NOT REAL BIBLIOGRAPHIES, I JUST COPIED: 

N. Askin and M. Mauskapf, "Cultural Attributes and Their Influence on Consumption Patterns in Popular Music", Proceedings of the 6th International Conference SocInfo 2014, pp. 508-530, 2014.: (https://link.springer.com/chapter/10.1007/978-3-319-13734-6_36 )


https://royalsocietypublishing.org/doi/10.1098/rsos.171274#d1e854 
